<a href="https://colab.research.google.com/github/jisang93/NLP_Class/blob/master/16%20Deep%20Learning%20XOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **딥러닝을 이용한 XOR**

## **1. XOR**

In [235]:
import numpy as np

# X, Y 값
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])

X, Y

(array([[0, 0, 1, 1],
        [0, 1, 0, 1]]), array([[0, 1, 1, 0]]))

In [236]:
# h1, h2, h1_bias, h2_bias
h1_weight = np.array([[-0.89, 0.028], [0.098, -0.07]])
h1_bias = np.array([[0.092], [-0.01]])

h2_weight = np.array([[0.056], [0.067]])
h2_bias = np.array([[0.016]])

In [237]:
# X*Weight + Bias*Weight
h1_cal = np.dot(h1_weight, X) + h1_bias

h1_cal

array([[ 0.092,  0.12 , -0.798, -0.77 ],
       [-0.01 , -0.08 ,  0.088,  0.018]])

In [238]:
# 시그모이드 함수
def sigmoid(x):
	return 1 / (1 + np.exp(-x))
 
h1_sig = sigmoid(h1_cal)
h1_sig

array([[0.52298379, 0.52996405, 0.3104535 , 0.31647911],
       [0.49750002, 0.48001066, 0.52198581, 0.50449988]])

In [239]:
h2_cal = np.dot(h1_sig.T, h2_weight) + h2_bias
h2_cal

array([[0.07861959],
       [0.0778387 ],
       [0.06835845],
       [0.06752432]])

In [240]:
predict = sigmoid(h2_cal)
predict

array([[0.51964478],
       [0.51944986],
       [0.51708296],
       [0.51687467]])

In [241]:
diff = predict - Y.T
diff

array([[ 0.51964478],
       [-0.48055014],
       [-0.48291704],
       [ 0.51687467]])

In [242]:
def loss(r, p):
   return -((np.dot(r, np.log(p)) + np.dot((1 - r), np.log(1-p)))/len(r[0]))[0][0]

L = loss(Y, predict)
L

0.6938113835190924

In [243]:
grad_h2 = np.dot(h1_sig, diff)
grad_h2

array([[0.03074824],
       [0.03654146]])

In [244]:
alpha = 0.01
h2_weight = h2_weight - grad_h2 * alpha
h2_weight

array([[0.05569252],
       [0.06663459]])

In [245]:
h2_bias = h2_bias - alpha * diff
h2_bias

array([[0.01080355],
       [0.0208055 ],
       [0.02082917],
       [0.01083125]])

In [246]:
 h1_weight = h1_weight - alpha * np.dot(X, np.dot(np.dot(np.dot(h2_weight, diff.T).T, h1_sig), (1 - h1_sig).T))
 h1_weight

array([[-0.89004399,  0.02796144],
       [ 0.09795294, -0.07004125]])

In [247]:
h1_bias = h1_bias - np.dot(np.dot(np.dot(h2_weight, diff.T).T, h1_sig), (1 - h1_sig).T).T
h1_bias

array([[ 0.02468201,  0.15425343,  0.15456005,  0.02504087],
       [-0.06900864,  0.04456922,  0.04483799, -0.06869408]])

## **2. XOR 클래스화**

### **2-1. XOR 클래스**

In [230]:
import numpy as np

class XOR():
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        self.x2h = [] 
        self.x2h_bias = []
        self.h2y = []
        self.h2y_bias = []
        self.x2h_sig = []
        self.predict = []
        self.diff = []
        self.L = 0

    # Hidden Layer 설정
    def hidden_layer(self, k):
        # # X to Hidden Layer Weight
        self.x2h = np.random.rand(k, len(self.X))
        self.x2h_bias = np.random.rand(len(self.X),1)
        # Hidden Layer to Y Weight
        self.h2y = np.random.rand(len(self.X), len(self.Y))
        self.h2y_bias = np.random.rand(1, len(self.Y))

        return self.x2h, self.x2h_bias, self.h2y, self.h2y_bias

    # Y 예측
    def predict_Y(self):
        # Sigmoid 함수
        def sigmoid(x):
	        return 1 / (1 + np.exp(-x))
         
        # X * X2H_Weight + X2H_Bias
        x2h_cal = np.dot(self.x2h.T, X) + self.x2h_bias
        self.x2h_sig = sigmoid(x2h_cal)
        # Sigmoid_X * H2Y_Weight + H2Y_Bias
        h2y_cal = np.dot(self.x2h_sig.T, self.h2y) + self.h2y_bias
        self.predict = sigmoid(h2y_cal)
        self.diff = self.predict - self.Y.T

        # Loss 함수
        def loss(r, p):
            return -((np.dot(r, np.log(p)) + np.dot((1 - r), np.log(1-p)))/len(r[0]))[0][0]
        
        self.L = loss(self.Y, self.predict)

        return self.predict, self.L

    # Back Propagation 구현
    def back_propagation(self, alpha):
        grad_h2y = np.dot(self.x2h_sig, self.diff)
        # Hidden Layer to Y Weight Gradient 
        self.h2y = self.h2y - alpha * grad_h2y
        # Hidden layer to Y Bias Gradient
        self.h2y_bias = self.h2y_bias - alpha * self.diff
        # X to Hidden Layer Weight Gradient
        self.x2h = self.x2h - alpha * np.dot(X, np.dot(np.dot(np.dot(self.h2y, self.diff.T).T, self.x2h_sig), (1 - self.x2h_sig).T))
        # X to Hidden Layer Bias Gradient
        self.x2h_bias = self.x2h_bias - alpha * np.dot(np.dot(np.dot(self.h2y, self.diff.T).T, self.x2h_sig), (1-self.x2h_sig).T).T

    # 자동 실행
    def run(self, k, alpha, epochs):
        self.hidden_layer(k)
        L1 = 0
        for j in range(epochs):    
            self.predict_Y()
            if L1 != self.L:
                self.back_propagation(alpha)
                L1 = self.L.copy()
            else:
                break
        print("Loss : {}. 예측 Y값 : {}".format(self.L, self.predict.T))

### **2-2. 결과 확인**

In [231]:
# X, Y 값
X = np.array([[0, 0, 1, 1], [0, 1, 0, 1]])
Y = np.array([[0, 1, 1, 0]])

In [232]:
xor = XOR(X, Y)

In [233]:
xor.run(2, 0.01, 1000)

Loss : 0.0786985355921335. 예측 Y값 : [[0.05891401 0.90806575 0.90839195 0.0596993 ]]
